In [1]:
#install pyspark
!pip install pyspark

     |████████████████████████████████| 212.4 MB 63 kB/s 
     |████████████████████████████████| 198 kB 49.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=a0a7cf601d0cddc4343863d3b594bb88d4f924e0f71c5d4618164b80cfba1773
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [4]:
#create a sparksession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark").getOrCreate()

In [5]:
#clone the dataset
! git clone https://github.com/education454/admission_dataset

Cloning into 'admission_dataset'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [6]:
#check the presence of dataset
! ls admission_dataset

Admission_Predict_Ver1.1.csv


In [8]:
#create a spark dataframe
df = spark.read.csv('/content/admission_dataset/Admission_Predict_Ver1.1.csv',header =True, inferSchema =True)

In [9]:
#display dataframe
df.show()

+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|Serial No|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|        1|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|        2|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|        3|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|        4|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|        5|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|        6|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|        7|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|        8|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|        9

In [10]:
#get the no.of rows & columns
print((df.count(), len(df.columns)))

(500, 9)


In [11]:
#print schema 
df.printSchema()

root
 |-- Serial No: integer (nullable = true)
 |-- GRE Score: integer (nullable = true)
 |-- TOEFL Score: integer (nullable = true)
 |-- University Rating: integer (nullable = true)
 |-- SOP: double (nullable = true)
 |-- LOR: double (nullable = true)
 |-- CGPA: double (nullable = true)
 |-- Research: integer (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [13]:
#get the summary statistics
df.describe().show()

+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|summary|        Serial No|         GRE Score|      TOEFL Score|University Rating|               SOP|               LOR|              CGPA|          Research|    Chance of Admit|
+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|  count|              500|               500|              500|              500|               500|               500|               500|               500|                500|
|   mean|            250.5|           316.472|          107.192|            3.114|             3.374|             3.484| 8.576440000000003|              0.56| 0.7217399999999996|
| stddev|144.4818327679989|11.295148372354712|6.081867659564538|1.143511800759815|0.9910036207566072|0.92

In [14]:
#drop the unnecessary column
df = df.drop('Serial No')

In [15]:
#display the dataframe
df.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|      302|        102|                1|2.0|1.5| 8.0|       0|            0.5|
|      323|        108|                3

In [20]:
#check for null values
for i in df.columns:
  print(i+":",df[df[i].isNull()].count())


GRE Score: 0
TOEFL Score: 0
University Rating: 0
SOP: 0
LOR: 0
CGPA: 0
Research: 0
Chance of Admit: 0


In [22]:
# correlation analysis
for col in df.columns:
  print('Correlation to chance of admit col for {} is {}'.format(col,df.stat.corr('Chance of Admit',col)))

Correlation to chance of admit col for GRE Score is 0.8103506354632598
Correlation to chance of admit col for TOEFL Score is 0.7922276143050823
Correlation to chance of admit col for University Rating is 0.6901323687886892
Correlation to chance of admit col for SOP is 0.6841365241316723
Correlation to chance of admit col for LOR is 0.6453645135280112
Correlation to chance of admit col for CGPA is 0.882412574904574
Correlation to chance of admit col for Research is 0.5458710294711379
Correlation to chance of admit col for Chance of Admit is 1.0


In [24]:
# feature selection
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['GRE Score', 'TOEFL Score', 'CGPA'],outputCol='features')

In [25]:
#display dataframe
output_data = assembler.transform(df)
output_data.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|          features|
+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|[337.0,118.0,9.65]|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|[324.0,107.0,8.87]|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72| [316.0,104.0,8.0]|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|[322.0,110.0,8.67]|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|[314.0,103.0,8.21]|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|[330.0,115.0,9.34]|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75| [321.0,109.0,8.2]|
|      308

In [28]:
#import linear regression and create final data
from pyspark.ml.regression import LinearRegression
final_data = output_data.select('features','Chance of Admit')

In [29]:
#print schema of final data
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [30]:
#split the dataset into train and test
train, test = final_data.randomSplit([0.7,0.3])

In [31]:
#build & train the model
models = LinearRegression(featuresCol='features', labelCol= 'Chance of Admit')
model = models.fit(train)

In [32]:
#get coefficients & intercept
print('Coefficients:',model.coefficients)
print('Intercept:', model.intercept)

Coefficients: [0.0018837939575061357,0.003754261945529647,0.14877344159079597]
Intercept: -1.5559044437448364


In [33]:
#get summary of the model
summary = model.summary

In [34]:
#print the rmse & r2 score
print('RMSE:', summary.rootMeanSquaredError)
print('R2:', summary.r2)

RMSE: 0.0666242361844099
R2: 0.7852936348728586


In [35]:
#transform on the test data
predictions = model.transform(test)

In [36]:
#display the predictions
predictions.show()

+------------------+---------------+-------------------+
|          features|Chance of Admit|         prediction|
+------------------+---------------+-------------------+
|[290.0,100.0,7.56]|           0.47|  0.490549216911325|
|[290.0,104.0,7.46]|           0.45|0.49068892053436386|
| [295.0,96.0,7.34]|           0.47| 0.4522209817667622|
| [296.0,99.0,8.03]|           0.61| 0.5680212362585064|
| [297.0,96.0,7.43]|           0.34|0.46937817942494586|
| [297.0,98.0,7.67]|           0.59| 0.5125923292977963|
| [298.0,97.0,7.21]|           0.45| 0.4422860781780067|
|[298.0,101.0,7.69]|           0.53| 0.5287143779237073|
| [299.0,96.0,7.86]|           0.54| 0.5371183472240006|
|[299.0,100.0,7.89]|           0.59| 0.5565985982538431|
|[299.0,102.0,8.62]|           0.56| 0.6727117345061835|
| [300.0,95.0,8.22]|           0.62| 0.5888063182086638|
|  [300.0,99.0,6.8]|           0.36|  0.392565078931852|
| [300.0,105.0,7.8]|           0.58| 0.5638640921958258|
| [301.0,96.0,7.56]|           

In [39]:
#evaluate the model
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(predictionCol = 'prediction', labelCol ='Chance of Admit',metricName='r2')
print('r2 on the test data:', evaluator.evaluate(predictions))

r2 on the test data: 0.8521622881497064


In [40]:
#save the model
model.save('model')

In [41]:
#load the model
from pyspark.ml.regression import LinearRegressionModel
model = LinearRegressionModel.load('model')